# Dynamic inter-prism calibration notebook

Notebook to test the dynamic inter-prism calibration with one set of data

In [ ]:
import numpy as np
import importlib
import matplotlib.pyplot as plt
import scipy
import warnings
warnings.filterwarnings('ignore')
import scripts.theodolite_utils as theodo_u
import scripts.resection_functions as theodo_r

# Input for calibration

In [ ]:
# Read predicted prism trajectories
theodo_u = importlib.reload(theodo_u)

# path = "../data/20220717-2/"
# path_type = "filtered_prediction/"
# file_name = "f-2-1-1-1-6-0-L_"

path = "../data/Simulation_1/"
path_type = "total_stations/"
file_name = "ns_sync_p_"

trimble_1 = np.array(theodo_u.read_prediction_data_Linear_csv_file(path+path_type+file_name+"1.csv"))
trimble_2 = np.array(theodo_u.read_prediction_data_Linear_csv_file(path+path_type+file_name+"2.csv"))
trimble_3 = np.array(theodo_u.read_prediction_data_Linear_csv_file(path+path_type+file_name+"3.csv"))
print("Number of points: ", len(trimble_1),len(trimble_2),len(trimble_3))

In [ ]:
# Display prism trajectories
%matplotlib notebook
plt.figure(figsize=(10,3))
plt.scatter(trimble_1[:,1],trimble_1[:,2], color='r')
plt.scatter(trimble_2[:,1],trimble_2[:,2], color='b')
plt.scatter(trimble_3[:,1],trimble_3[:,2], color='g')
plt.show()

# Prism positions selection

In [ ]:
# Sort points according to their speed
theodo_u = importlib.reload(theodo_u)
rate = 2.5
speed_limit = 10
index1 = theodo_u.find_not_moving_points_GP(np.array(trimble_1), speed_limit, 1/rate)

In [ ]:
# Split set of positions selected into a training set and an evalution set
theodo_u = importlib.reload(theodo_u)

p1s_l = trimble_1[index1,1:5]
p2s_l = trimble_2[index1,1:5]
p3s_l = trimble_3[index1,1:5]

threshold_training = 0.75
mask_1 = theodo_u.random_splitting_mask(p1s_l, threshold_training)

# training set
p1s_t = p1s_l[mask_1]
p2s_t = p2s_l[mask_1]
p3s_t = p3s_l[mask_1]

# evaluation set
p1s_p = p1s_l[~mask_1]
p2s_p = p2s_l[~mask_1]
p3s_p = p3s_l[~mask_1]

print("Number of points kept: ", len(p1s_l),len(p1s_t),len(p1s_p))

In [ ]:
# Information about the shape of the point cloud
cov_1 = np.cov(p1s_t[:,0:3].T)
w1, v1 = np.linalg.eig(cov_1)
print("Eigen values: ", (w1)**0.5)

# Prior computation

In [ ]:
# Compute the prior
theodo_u = importlib.reload(theodo_u)
theodo_r = importlib.reload(theodo_r)

method_prior = "Else"    #GCP or PTP or None
method_6DOF = True

if(method_prior == "GCP"):
    file_name_marker = [path+"total_stations/GCP.txt"]
    marker_1, marker_2, marker_3, T1_GCP, T12_GCP, T13_GCP = theodo_u.read_marker_file(file_name_marker[0], 1, 1)
    T12_init = T12_GCP
    T13_init = T13_GCP
else:
    if(method_prior == "PTP"):
        T12_init = theodo_u.point_to_point_minimization(p2s_t.T, p1s_t.T)
        T13_init = theodo_u.point_to_point_minimization(p3s_t.T, p1s_t.T)
    else:
        if(method_prior == "None"):
            T12_init=np.identity(4)
            T13_init=np.identity(4)
        else:
            T12_init = np.array([[-0.0691160,  0.4427109,  0.8939967, 10],[0.5290934, -0.7434545,  0.4090667, -5],
                                 [0.8457441,  0.5012808, -0.1828508, -3],[0,0,0,1]])
            T13_init = np.array([[-0.0691160, -0.4427109,  0.8939967, -6],[-0.5290934, -0.7434545, -0.4090667, 4],
                                 [0.8457441, -0.5012808, -0.1828508, 1],[0,0,0,1]])

T12_init_log = theodo_r.exp_inv_T(T12_init)
T13_init_log = theodo_r.exp_inv_T(T13_init)
if(method_6DOF):
    x_init = [T12_init_log[2,1],T12_init_log[0,2],T12_init_log[1,0],T12_init_log[0,3],T12_init_log[1,3],T12_init_log[2,3],
          T13_init_log[2,1],T13_init_log[0,2],T13_init_log[1,0],T13_init_log[0,3],T13_init_log[1,3],T13_init_log[2,3]]
else:
    x_init = [T12_init_log[1,0],T12_init_log[0,3],T12_init_log[1,3],T12_init_log[2,3],
             T13_init_log[1,0],T13_init_log[0,3],T13_init_log[1,3],T13_init_log[2,3]]

pt2l = (T12_init@p2s_l.T).T
pt3l = (T13_init@p3s_l.T).T

print(x_init)

In [ ]:
# Display the prism positions aligned with the prior selected
%matplotlib notebook
plt.figure(figsize=(10,3))
plt.scatter(p1s_l[:,0],p1s_l[:,1], color='b')
plt.scatter(pt2l[:,0],pt2l[:,1], color='r')
plt.scatter(pt3l[:,0],pt3l[:,1], color='g')
plt.show()

# Minimization

In [ ]:
# Minimization with the cost function
theodo_u = importlib.reload(theodo_u)
theodo_r = importlib.reload(theodo_r)

import time

extrinsic_calibration_results = theodo_u.read_extrinsic_calibration_results_file(path+"sensors_extrinsic_calibration/calibration_results.csv")

start_time = time.time()

if(method_6DOF):
    res = scipy.optimize.least_squares(lambda x: theodo_r.cost_fun_ls(p1s_t, p2s_t, p3s_t, x[:6], x[6:], extrinsic_calibration_results[0],      extrinsic_calibration_results[1], extrinsic_calibration_results[2]), x0=x_init, method='lm', ftol=1e-15, xtol=1e-15, gtol=1e-15, x_scale=1.0,
                                       loss='linear', f_scale=1.0, diff_step=None, tr_solver=None, tr_options={}, jac_sparsity=None, max_nfev=30000000,
                                       verbose=2, args=(), kwargs={})
    xi_12 = res.x[:6]
    xi_13 = res.x[6:]
    T12 = theodo_r.exp_T(xi_12)
    T13 = theodo_r.exp_T(xi_13)
else:
    res = scipy.optimize.least_squares(lambda x: theodo_r.cost_fun_ls_4dof(p1s_t, p2s_t, p3s_t, x[:4], x[4:], extrinsic_calibration_results[0],      extrinsic_calibration_results[1], extrinsic_calibration_results[2]), x0=x_init, method='trf', ftol=1e-15, xtol=1e-15, gtol=1e-15, x_scale=1.0,
                                       loss='linear', f_scale=1.0, diff_step=None, tr_solver=None, tr_options={}, jac_sparsity=None, max_nfev=30000000,
                                       verbose=2, args=(), kwargs={})
    xi_12 = res.x[:4]
    xi_13 = res.x[4:]
    T12 = theodo_r.exp_T_4dof(xi_12)
    T13 = theodo_r.exp_T_4dof(xi_13)

stop_time = time.time()
print(stop_time - start_time)
print(res.fun)

# Analysis

In [ ]:
# Difference between the prior and the results
theodo_u = importlib.reload(theodo_u)
theodo_r = importlib.reload(theodo_r)

p1s_r = p1s_p
p2s_r = (T12@p2s_p.T).T
p3s_r = (T13@p3s_p.T).T

save_tf = False
name_file_type = 'f-2-1-1-1-6-1-L'
if(save_tf):
    theodo_u.save_tf(np.identity(4), T12, T13, path+"list_tf/dynamic_inter_prism_test-"+name_file_type+".csv")

A_12 = np.array([theodo_r.exp_inv_T(T12)[2,1],theodo_r.exp_inv_T(T12)[0,2],theodo_r.exp_inv_T(T12)[1,0],theodo_r.exp_inv_T(T12)[0,3],theodo_r.exp_inv_T(T12)[1,3],theodo_r.exp_inv_T(T12)[2,3]])
A_13 =  np.array([theodo_r.exp_inv_T(T13)[2,1],theodo_r.exp_inv_T(T13)[0,2],theodo_r.exp_inv_T(T13)[1,0],theodo_r.exp_inv_T(T13)[0,3],theodo_r.exp_inv_T(T13)[1,3],theodo_r.exp_inv_T(T13)[2,3]])
B_12 =  np.array([theodo_r.exp_inv_T(T12_init)[2,1],theodo_r.exp_inv_T(T12_init)[0,2],theodo_r.exp_inv_T(T12_init)[1,0],theodo_r.exp_inv_T(T12_init)[0,3],theodo_r.exp_inv_T(T12_init)[1,3],theodo_r.exp_inv_T(T12_init)[2,3]])
B_13 =  np.array([theodo_r.exp_inv_T(T13_init)[2,1],theodo_r.exp_inv_T(T13_init)[0,2],theodo_r.exp_inv_T(T13_init)[1,0],theodo_r.exp_inv_T(T13_init)[0,3],theodo_r.exp_inv_T(T13_init)[1,3],theodo_r.exp_inv_T(T13_init)[2,3]])
print("Difference with prior 12: ",A_12-B_12)
print("Difference with prior 13: ",A_13-B_13)

In [ ]:
# Errors obtained for the different metrics
dist_prism_new = []
dist_prism_basic = []
timestamp = np.array(trimble_1)[:,0]
for i in range(0,len(p1s_r)-1):
    dp1 = abs(np.linalg.norm(p1s_r[i,0:3]-p2s_r[i,0:3])-extrinsic_calibration_results[0])*1000
    dp2 = abs(np.linalg.norm(p1s_r[i,0:3]-p3s_r[i,0:3])-extrinsic_calibration_results[1])*1000
    dp3 = abs(np.linalg.norm(p3s_r[i,0:3]-p2s_r[i,0:3])-extrinsic_calibration_results[2])*1000
    dist_prism_new.append(np.array([timestamp[i], dp1, dp2, dp3]))
    
    dp1 = abs(np.linalg.norm(p1s_l[i,0:3]-(T12_GCP@p2s_l[i,0:4].T)[0:3])-extrinsic_calibration_results[0])*1000
    dp2 = abs(np.linalg.norm(p1s_l[i,0:3]-(T13_GCP@p3s_l[i,0:4].T)[0:3])-extrinsic_calibration_results[1])*1000
    dp3 = abs(np.linalg.norm((T13_GCP@p3s_l[i,0:4].T)[0:3]-(T12_GCP@p2s_l[i,0:4].T)[0:3])-extrinsic_calibration_results[2])*1000
    dist_prism_basic.append(np.array([timestamp[i], dp1, dp2, dp3]))
dist_prism_new = np.array(dist_prism_new)
dist_prism_basic = np.array(dist_prism_basic)

print("Inter-prism distance error metric")
print("GCP")
print("Mean inter-prism [mm]: ", round(np.mean(dist_prism_basic[:,1]),2), round(np.mean(dist_prism_basic[:,2]),2), round(np.mean(dist_prism_basic[:,3]),2))
print("Std inter-prism [mm]: ", round(np.std(dist_prism_basic[:,1]),2), round(np.std(dist_prism_basic[:,2]),2), round(np.std(dist_prism_basic[:,3]),3))
print("Dynamic inter-prism")
print("Mean inter-prism [mm]: ", round(np.mean(dist_prism_new[:,1]),2), round(np.mean(dist_prism_new[:,2]),2), round(np.mean(dist_prism_new[:,3]),2))
print("Std inter-prism [mm]: ", round(np.std(dist_prism_new[:,1]),2), round(np.std(dist_prism_new[:,2]),2), round(np.std(dist_prism_new[:,3]),3))

In [ ]:
# Display inter-prism metric
print("Mean inter-prism [mm]: ", round(np.mean(dist_prism_new[:,1]),2), round(np.mean(dist_prism_new[:,2]),2), round(np.mean(dist_prism_new[:,3]),2))
print("Std inter-prism [mm]: ", round(np.std(dist_prism_new[:,1]),2), round(np.std(dist_prism_new[:,2]),2), round(np.std(dist_prism_new[:,3]),3))
plt.figure(figsize=(10,5))
offset_timestamp = min(dist_prism_new[:,0])
plt.subplot(111)
plt.plot(dist_prism_new[:,0]-offset_timestamp, dist_prism_new[:,1], label="P12", color="red", alpha=0.5)
plt.plot(dist_prism_new[:,0]-offset_timestamp, dist_prism_new[:,2], label="P13", color="blue", alpha=0.5)
plt.plot(dist_prism_new[:,0]-offset_timestamp, dist_prism_new[:,3], label="P23", color="green", alpha=0.5)
ax = plt.gca()
ax.set_yscale('symlog')
plt.legend(loc='best')
ax.set_ylabel("Error [mm]")
#limitx = [-50, 350]
#ax.set_xlim(limitx)
ax.set_xlabel("Time [s] \n Mean inter-prism [mm]: " + str(round(np.mean(dist_prism_new[:,1]),2)) + "  " + str(round(np.mean(dist_prism_new[:,2]),2)) + "  " + str(round(np.mean(dist_prism_new[:,3]),2))
             + "\n Std inter-prism [mm]: " + str(round(np.std(dist_prism_new[:,1]),2)) + "  " + str(round(np.std(dist_prism_new[:,2]),2)) + "  " + str(round(np.std(dist_prism_new[:,3]),2)))
plt.tight_layout()
plt.show()  
plt.savefig(path+"figures/dynamic_inter_prism_test"+name_file_type+".jpg")

# Trajectories analysis

In [ ]:
# Display evaluation prism positions
%matplotlib notebook
p2s_rarr = np.array(p2s_r)
p3s_rarr = np.array(p3s_r)
plt.figure(figsize=(10,3))
plt.scatter(p1s_r[:,0],p1s_r[:,1], color='r')
plt.scatter(p2s_rarr[:,0],p2s_rarr[:,1], color='b')
plt.scatter(p3s_rarr[:,0],p3s_rarr[:,1], color='g')
plt.show()

In [ ]:
# Dispay X axis
t1s_r = trimble_1[:,1:5]
t2s_r = (T12@(trimble_2[:,1:5]).T).T
t3s_r = (T13@(trimble_3[:,1:5]).T).T
plt.figure(figsize=(10,3))
offset_timestamp = min(trimble_1[0,0],trimble_2[0,0],trimble_3[0,0])
plt.plot(trimble_1[:,0]-offset_timestamp,t1s_r[:,0], color='r')
plt.plot(trimble_2[:,0]-offset_timestamp,t2s_r[:,0], color='b')
plt.plot(trimble_3[:,0]-offset_timestamp,t3s_r[:,0], color='g')
plt.show()

In [ ]:
# Dispay Y axis
plt.figure(figsize=(10,3))
plt.plot(trimble_1[:,0]-offset_timestamp,t1s_r[:,1], color='r')
plt.plot(trimble_2[:,0]-offset_timestamp,t2s_r[:,1], color='b')
plt.plot(trimble_3[:,0]-offset_timestamp,t3s_r[:,1], color='g')
plt.show()

In [ ]:
# Dispay Z axis
plt.figure(figsize=(10,3))
plt.plot(trimble_1[:,0]-offset_timestamp,t1s_r[:,2], color='r')
plt.plot(trimble_2[:,0]-offset_timestamp,t2s_r[:,2], color='b')
plt.plot(trimble_3[:,0]-offset_timestamp,t3s_r[:,2], color='g')
plt.show()

# Analysis with GCP

In [ ]:
m1_b = marker_1
m2_b = T12_GCP@marker_2
m3_b = T13_GCP@marker_3
m1_n = marker_1
m2_n = T12@marker_2
m3_n = T13@marker_3

In [ ]:
error_basic = []
error_new = []
for i,j,k in zip(m1_b.T, m2_b.T, m3_b.T):
    dist_12 = np.linalg.norm(i[0:3]-j[0:3])*1000
    dist_13 = np.linalg.norm(i[0:3]-k[0:3])*1000
    dist_23 = np.linalg.norm(k[0:3]-j[0:3])*1000
    error_basic.append(dist_12)
    error_basic.append(dist_13)
    error_basic.append(dist_23)
            
for i,j,k in zip(m1_n.T, m2_n.T, m3_n.T):
    dist_12 = np.linalg.norm(i[0:3]-j[0:3])*1000
    dist_13 = np.linalg.norm(i[0:3]-k[0:3])*1000
    dist_23 = np.linalg.norm(k[0:3]-j[0:3])*1000
    error_new.append(dist_12)
    error_new.append(dist_13)
    error_new.append(dist_23)

print("Ground control points error metric [mm]")
print("Number control points: ", len(marker_1.T))
print("GCP: ",np.median(error_basic))
print("Dynamic inter-prism: ",np.median(error_new))

In [ ]:
# Display GCP with prism trajectories
plt.figure(figsize=(10,3))
axis_1 = 0
axis_2 = 1
plt.plot(p1s_r[:,axis_1],p1s_r[:,axis_2], color='r')
plt.plot(p2s_r[:,axis_1],p2s_r[:,axis_2], color='b')
plt.plot(p3s_r[:,axis_1],p3s_r[:,axis_2], color='g')
plt.scatter(m1_b[axis_1,:],m1_b[axis_2,:], color='black')
plt.scatter(m2_b[axis_1,:],m2_b[axis_2,:], color='black')
plt.scatter(m3_b[axis_1,:],m3_b[axis_2,:], color='black')
plt.scatter(m1_n[axis_1,:],m1_n[axis_2,:], color='brown')
plt.scatter(m2_n[axis_1,:],m2_n[axis_2,:], color='orange')
plt.scatter(m3_n[axis_1,:],m3_n[axis_2,:], color='yellow')
plt.xlabel('X [m]')
plt.ylabel('Y [m]')
plt.tight_layout()
#plt.savefig("./figs/20220505_empty_YZ_plan.jpg")
plt.show()